In [1]:
import cv2
import numpy as np
#used for distance calculation
from sklearn.metrics import pairwise

## GLOBAL VARIABLES

In [2]:
# This background will be a global variable that we update through a few functions
background=None
accumulated_weight = 0.5

#Manually set up our ROI for grabbing the hand
#corners of the rectangle 
ROI_top=20
ROI_bottom=300
ROI_right=300
ROI_left=600

## Finding Average Background Value

In [3]:
#function to find average background value
def calc_accum_avg(frame,accumulated_weight):
    global background
    
    #creating the background from a copy of the frame
    if background is None:
        background=frame.copy().astype("float")
        return None
    #computing the weighted average,accumulate it and update the background
    cv2.accumulateWeighted(frame,background,accumulated_weight) 

## Segment the hand region in frame

In [4]:
def segment(frame,threshold = 34):
    global background
    
    #calculating the Absolute difference between the background and the passed in frame
    diff = cv2.absdiff(background.astype("uint8"),frame)
    
    #Applying a threshold to the image so we can grab the foreground
    ret,thresholded = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    
    #grabbing the external contours from the image
    contours,hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    #returning None coz we didnt grab any contours 
    if len(contours) == 0:
        return None
    else:
        #Assuming the largest external contour in roi is the hand
        #This will be our segment
        hand_segment = max(contours,key=cv2.contourArea)
        
        #returning the hand segment and the thresolded hand image
        return (thresholded,hand_segment)

## Counting Fingers with a convex hull
https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html#:~:text=5.-,Convex%20Hull,-Convex%20Hull%20will

Refer the Above link for more info on convex hull

In [5]:
def count_fingers(thresholded,hand_segment):
    
    #calculated the convex hull of the hand segment
    conv_hull=cv2.convexHull(hand_segment)
    
    #finding the top,bottom,left,right and making sure that they are in a tuple format
    top   =tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom=tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left  =tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right =tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    cX=(left[0]+right[0]) // 2 #0 coz the left and right is a tuple of x,y coordinates and we want the x coordinates for cX
    cY=(top[1]+bottom[1]) // 2 #0 coz the top and bottom is a tuple of x,y coordinates and we want the y coordinate for cY
    
    #calculating the euclidean distance between the center of hand and left,right,top,bottom points
    distance=pairwise.euclidean_distances([(cX,cY)],Y=[left,right,top,bottom])[0]
    
    #grabbing the largest distance
    max_distance=distance.max()
    
    #creating the circle with 75% radius of the max euclidean distance
    radius=int(0.75*max_distance)#adjust the % according to ur hand size
    circumference=(2*np.pi*radius)
    
    #grabbing roi of that circle
    circular_roi=np.zeros(thresholded.shape[:2],dtype='uint8')
    
    #draw the circle
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    #using bitwise AND with the circle ROI as a mask
    circular_roi=cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    #grabbing contours in circle ROI
    contours,hierarchy=cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    #Finger count starts at 0
    count=0
    
    # loop through the contours to see if we count any more fingers.
    for cnt in contours:
        # Bounding box of countour
        (x,y,w,h)=cv2.boundingRect(cnt)
#       Increment count of fingers based on two conditions
        
# 1. Contour region is not the very bottom of hand area (the wrist)
        out_of_wrist=((cY+(cY*0.25)) > (y+h))
    
# 2. Number of points along the contour does not exceed 25% of the circumference of the circular ROI (otherwise we're counting points off the hand)
        limit_points=((circumference*0.25)>cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count+=1
    return count

In [6]:
cam=cv2.VideoCapture(0)
num_frames=0
while True:
    ret,frame=cam.read()
    #flipping the frame so that its not in the mirros view
    frame = cv2.flip(frame, 1)
    frame_copy=frame.copy()
    #grabbing roi from the frame
    roi=frame[ROI_top:ROI_bottom,ROI_right:ROI_left]
   
    #Appy grayscale and blur to ROI
    gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(7,7),0)
    
# For the first 30 frames we will calculate the average of the background.
    if num_frames<60:
        calc_accum_avg(gray,accumulated_weight)
        if num_frames<=59:
            cv2.putText(frame_copy,'WAIT. GETTING BACKGROUND',(200,400),cv2.FONT_HERSHEY_TRIPLEX,1,(0,255,0),2)
            cv2.imshow('FINGER COUNT',frame_copy)
    else:
        #segmenting the hand as we have the background
        hand=segment(gray)
        #checking if we are able to detect the hand
        if hand is not None:
            thresholded,hand_segment=hand
            #draw contours around real hand in live stream
            cv2.drawContours(frame_copy,[hand_segment+(ROI_right,ROI_top)],-1,(255,0,255),1)
            #count the fingers
            fingers=count_fingers(thresholded,hand_segment)
            #display the count
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SCRIPT_COMPLEX,1,(0,0,255),2)
            #Also disply the thresolded image
            cv2.imshow('Thresholded',thresholded)

    # Draw ROI Rectangle on frame copy
    cv2.rectangle(frame_copy,(ROI_left,ROI_top),(ROI_right,ROI_bottom),(0,0,255),5)
    #increment the number of frames for tracking
    num_frames+=1
    #disply the frame with segmented hand
    cv2.imshow('Finger count',frame_copy)
    
    #closing the window with esc key
    k=cv2.waitKey(1) & 0xFF
    if k==27:
        break
        
# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()